In [ ]:
!pip install -U tensorflow_hub
!pip install -U tensorflow_datasets

In [ ]:
a = []
while 1:
    a.append("1")

In [ ]:
import time
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
# tfds.disable_progress_bar()

from tensorflow.keras import layers

In [ ]:
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
def format_image(image, label):
  # `hub` image modules exepct their data normalized to the [0,1] range.
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

In [ ]:
feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])

model.summary()

In [ ]:
model.compile(
  optimizer='adam', 
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 1
history = model.fit(train_batches, 
                    epochs=EPOCHS, 
                    validation_data=validation_batches)

In [ ]:
"""
Checking Predictions
"""
class_names = np.array(info.features['label'].names)
print(f"class names {class_names}")

# Run an image batch through the model and convert the indices to class names.

image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
print(f"predicted_class_names : {predicted_class_names}")

print("Labels: ", label_batch)
print("Predicted labels: ", predicted_ids)


plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

In [ ]:
"""
Save as Keras .h5 model
"""

t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

!ls


In [ ]:
"""
Load the Keras .h5 Model
"""

reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

In [ ]:
"""
Difference between the loaded model and previous model
"""

result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

(abs(result_batch - reloaded_result_batch)).max()

In [ ]:
# Keep Training

EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 775s 1s/step - loss: 0.0224 - accuracy: 0.9918 - val_loss: 0.0309 - val_accuracy: 0.9910
Epoch 2/3
582/582 [==============================] - 767s 1s/step - loss: 0.0190 - accuracy: 0.9934 - val_loss: 0.0304 - val_accuracy: 0.9912
Epoch 3/3
582/582 [==============================] - 767s 1s/step - loss: 0.0173 - accuracy: 0.9945 - val_loss: 0.0316 - val_accuracy: 0.9910


In [ ]:
# Export as SavedModel

t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

!ls {export_path_sm}

./1617186785
INFO:tensorflow:Assets written to: ./1617186785/assets


INFO:tensorflow:Assets written to: ./1617186785/assets


assets	saved_model.pb	variables


In [ ]:
"""
Load SavedModel
"""

reloaded_sm = tf.saved_model.load(export_path_sm)
reload_sm_result_batch = reloaded_sm(image_batch, training=False).numpy()
(abs(result_batch - reload_sm_result_batch)).max()

In [ ]:
"""
Loading the SavedModel as a Keras Model
"""
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)
tf.saved_model.save(model, export_path_sm)

reload_sm_keras = tf.keras.models.load_model(
  export_path_sm,
  custom_objects={'KerasLayer': hub.KerasLayer})

reload_sm_keras.summary()

result_batch = model.predict(image_batch)
reload_sm_keras_result_batch = reload_sm_keras.predict(image_batch)

(abs(result_batch - reload_sm_keras_result_batch)).max()


In [ ]:
"""
Download your model
"""
!zip -r model.zip {export_path_sm}
!ls

try:
  from google.colab import files
  files.download('./model.zip')
except ImportError:
  pass